## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**1107. New Users Daily Count (Medium)**

**Table: Traffic**

| Column Name   | Type    |
|---------------|---------|
| user_id       | int     |
| activity      | enum    |
| activity_date | date    |

This table may have duplicate rows.
The activity column is an ENUM (category) type of ('login', 'logout', 'jobs', 'groups', 'homepage').
 
**Write a solution to reports for every date within at most 90 days from today, the number of users that logged in for the first time on that date. Assume today is 2019-06-30.**

Return the result table in any order.

The result format is in the following example.

**Example 1:**

**Input:** 

**Traffic table:**

| user_id | activity | activity_date |
|---------|----------|---------------|
| 1       | login    | 2019-05-01    |
| 1       | homepage | 2019-05-01    |
| 1       | logout   | 2019-05-01    |
| 2       | login    | 2019-06-21    |
| 2       | logout   | 2019-06-21    |
| 3       | login    | 2019-01-01    |
| 3       | jobs     | 2019-01-01    |
| 3       | logout   | 2019-01-01    |
| 4       | login    | 2019-06-21    |
| 4       | groups   | 2019-06-21    |
| 4       | logout   | 2019-06-21    |
| 5       | login    | 2019-03-01    |
| 5       | logout   | 2019-03-01    |
| 5       | login    | 2019-06-21    |
| 5       | logout   | 2019-06-21    |

**Output:** 
| login_date | user_count  |
|------------|-------------|
| 2019-05-01 | 1           |
| 2019-06-21 | 2           |

**Explanation:** 
Note that we only care about dates with non zero user count.
The user with id 5 first logged in on 2019-03-01 so he's not counted on 2019-06-21.

In [0]:
traffic_data_1107 = [
    (1, "login", "2019-05-01"),
    (1, "homepage", "2019-05-01"),
    (1, "logout", "2019-05-01"),
    (2, "login", "2019-06-21"),
    (2, "logout", "2019-06-21"),
    (3, "login", "2019-01-01"),
    (3, "jobs", "2019-01-01"),
    (3, "logout", "2019-01-01"),
    (4, "login", "2019-06-21"),
    (4, "groups", "2019-06-21"),
    (4, "logout", "2019-06-21"),
    (5, "login", "2019-03-01"),
    (5, "logout", "2019-03-01"),
    (5, "login", "2019-06-21"),
    (5, "logout", "2019-06-21"),
]

traffic_columns_1107 = ["user_id", "activity", "activity_date"]
traffic_df_1107 = spark.createDataFrame(traffic_data_1107, traffic_columns_1107)
traffic_df_1107.show()

+-------+--------+-------------+
|user_id|activity|activity_date|
+-------+--------+-------------+
|      1|   login|   2019-05-01|
|      1|homepage|   2019-05-01|
|      1|  logout|   2019-05-01|
|      2|   login|   2019-06-21|
|      2|  logout|   2019-06-21|
|      3|   login|   2019-01-01|
|      3|    jobs|   2019-01-01|
|      3|  logout|   2019-01-01|
|      4|   login|   2019-06-21|
|      4|  groups|   2019-06-21|
|      4|  logout|   2019-06-21|
|      5|   login|   2019-03-01|
|      5|  logout|   2019-03-01|
|      5|   login|   2019-06-21|
|      5|  logout|   2019-06-21|
+-------+--------+-------------+



In [0]:
login_df_1107 = traffic_df_1107.filter(col("activity") == "login")
login_df_1107.show()

+-------+--------+-------------+
|user_id|activity|activity_date|
+-------+--------+-------------+
|      1|   login|   2019-05-01|
|      2|   login|   2019-06-21|
|      3|   login|   2019-01-01|
|      4|   login|   2019-06-21|
|      5|   login|   2019-03-01|
|      5|   login|   2019-06-21|
+-------+--------+-------------+



In [0]:
first_login_df_1107 = login_df_1107.groupBy("user_id") \
                         .agg(min("activity_date").alias("first_login"))
first_login_df_1107.show()

+-------+-----------+
|user_id|first_login|
+-------+-----------+
|      1| 2019-05-01|
|      2| 2019-06-21|
|      3| 2019-01-01|
|      4| 2019-06-21|
|      5| 2019-03-01|
+-------+-----------+



In [0]:
first_login_df_1107\
    .filter((col("first_login") >= "2019-04-01") & (col("first_login") <= "2019-06-30"))\
        .groupBy("first_login") \
            .agg(count("user_id").alias("user_count")) \
                .withColumnRenamed("first_login", "login_date").show()

+----------+----------+
|login_date|user_count|
+----------+----------+
|2019-06-21|         2|
|2019-05-01|         1|
+----------+----------+

